In [170]:
## Clinic Booking Bot

##Easily book your clinic visit – available only on weekdays between **14:00 and 15:00**.  
##Speak or type, and get instant confirmation.


In [171]:
# import library

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from datetime import date
from PIL import Image, ImageDraw, ImageFont


In [172]:
# Save keys

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [173]:
# --- CONFIG ---
BOOKING_START = 14
BOOKING_END = 15
WEEKDAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
PHONE = "010-1234567"
confirmed_bookings = []


In [174]:
# --- TTS ---
def generate_tts(text, voice="fable", filename="output.mp3"):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="fable",
        input=text
    )
    with open(filename, "wb") as f:
        f.write(response.content)
    return filename

In [175]:
# --- Translate Booking Confirmation ---
def translate_text(text, target_language="nl"):
    prompt = f"Translate this message to {target_language}:\n{text}"
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful translator."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


In [176]:
# --- Booking Logic ---
def book_appointment(name, time_str):
    try:
        booking_time = datetime.strptime(time_str, "%H:%M")
    except ValueError:
        return "Invalid time format. Use HH:MM.", None, None

    hour = booking_time.hour
    weekday = datetime.today().strftime("%A")

    if weekday not in WEEKDAYS:
        response = "Bookings are only available on weekdays."
    elif BOOKING_START <= hour < BOOKING_END:
        confirmation = f"Booking confirmed for {name} at {time_str}."
        confirmed_bookings.append((name, time_str))
        translated = translate_text(confirmation)
        audio = generate_tts(translated)
        image = generate_booking_image(name, time_str)
        return translated, audio, image
    else:
        response = "Sorry, bookings are only accepted between 14:00 and 15:00 on weekdays."
        translated = translate_text(response)
        audio = generate_tts(translated)
        return translated, audio, None

In [177]:
# --- Booking Card ---
def generate_booking_image(name, time_str):
    img = Image.new("RGB", (500, 250), color="white")
    draw = ImageDraw.Draw(img)
    msg = f"\u2705 Booking Confirmed\nName: {name}\nTime: {time_str}"
    draw.text((50, 100), msg, fill="black")
    return img

In [178]:
# --- Voice Booking ---
def voice_booking(audio_path, name):
    with open(audio_path, "rb") as f:
        response = openai.audio.transcriptions.create(model="whisper-1", file=f)
    transcription = response.text.strip()

    system_prompt = """
    You are a clinic assistant. Extract only the appointment time from the user's sentence in 24-hour HH:MM format.
    If no time is mentioned, respond with 'No valid time found.'
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcription}
        ]
    )
    extracted_time = response.choices[0].message.content.strip()

    if ":" in extracted_time:
        return book_appointment(name, extracted_time)
    else:
        message = "Sorry, I couldn't understand the time. Please try again."
        translated = translate_text(message)
        audio_path = generate_tts(translated)
        return translated, audio_path, None

In [179]:
# --- Chat Bot Handler ---
def chat_bot(messages):
    system_prompt = """
    You are a clinic booking assistant. Your job is to:
    - Greet the patient and explain your role
    - Only assist with making appointments
    - Accept bookings only on weekdays between 14:00 and 15:00
    - Do not provide medical advice
    - Always respond with empathy and clarity
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": system_prompt}] + messages
    )
    reply = response.choices[0].message.content.strip()
    audio = generate_tts(reply)
    return reply, audio

In [180]:
# Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""## 🩺 GP Booking Assistant  
Only available weekdays between **14:00 and 15:00**  
☎️ Contact: {PHONE}
---""")

    name_global = gr.Textbox(label="Your Name", placeholder="Enter your name", interactive=True)

    with gr.Tab("💬 Chat Mode"):
        chatbot = gr.Chatbot(label="Booking Chat", type="messages", height=400)
        text_input = gr.Textbox(label="Type your message or use your voice below")
        audio_input = gr.Audio(type="filepath", label="🎙️ Or speak your request")
        chat_audio_output = gr.Audio(label="🔊 Assistant's Reply", type="filepath")
        send_btn = gr.Button("Send")

        def handle_chat(user_message, chat_history):
            chat_history = chat_history or []
            chat_history.append({"role": "user", "content": user_message})
            reply, audio = chat_bot(chat_history)
            chat_history.append({"role": "assistant", "content": reply})
            return chat_history, "", audio

        def handle_audio_chat(audio_path, chat_history):
            with open(audio_path, "rb") as f:
                transcription = openai.audio.transcriptions.create(model="whisper-1", file=f).text.strip()
            return handle_chat(transcription, chat_history)

        send_btn.click(handle_chat, [text_input, chatbot], [chatbot, text_input, chat_audio_output])
        text_input.submit(handle_chat, [text_input, chatbot], [chatbot, text_input, chat_audio_output])
        audio_input.change(handle_audio_chat, [audio_input, chatbot], [chatbot, text_input, chat_audio_output])


    
    with gr.Tab("📝 Text Booking"):
        time_text = gr.Textbox(label="Preferred Time (HH:MM)", placeholder="e.g., 14:30")
        btn_text = gr.Button("📅 Book via Text")

    with gr.Tab("🎙️ Voice Booking"):
        voice_input = gr.Audio(type="filepath", label="Say your preferred time")
        btn_voice = gr.Button("📅 Book via Voice")

    output_text = gr.Textbox(label="Response", interactive=False)
    output_audio = gr.Audio(label="Audio Reply", type="filepath")
    output_image = gr.Image(label="Booking Confirmation")

    btn_text.click(fn=book_appointment, inputs=[name_global, time_text], outputs=[output_text, output_audio, output_image])
    btn_voice.click(fn=voice_booking, inputs=[voice_input, name_global], outputs=[output_text, output_audio, output_image])

    gr.Markdown("""---
<small>This assistant does **not** give medical advice. It only books appointments within allowed hours.</small>
""")

    demo.launch()

* Running on local URL:  http://127.0.0.1:7898
* To create a public link, set `share=True` in `launch()`.
